In [1]:
import ee
import datetime
import folium
!pip install geemap
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AY0e-g6eapGadqCNM70R4h9_mhEQcNUuQK23pWDZ0pYgYnqjyInZKvaScn8

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
ee_date = ee.Date('2020-01-01')
py_date = datetime.datetime.utcfromtimestamp(ee_date.getInfo()['value']/1000.0)
print(py_date)

2020-01-01 00:00:00


In [5]:
medellin = ee.Image("users/rhodesailab3/Medellin_s2")
medellin_gt = ee.Image("users/rhodesailab3/Medellin_ground_truth_s2")
lower_gt = ee.Image("users/rhodesailab3/Kenya_Lower_30cm_ground_truth_dg")
makoko_gt = ee.Image("users/rhodesailab3/Makoko_50cm_large_ground_truth_dg")
mumbai_gt = ee.Image("users/rhodesailab3/Mumbai_ground_truth_s2")
kibera_gt = ee.Image("users/rhodesailab3/Kibera_30cm_ground_truth_dg")

print(medellin)
print(medellin.getInfo())

print(medellin_gt)
print(medellin_gt.getInfo())

print(lower_gt)
print(lower_gt.getInfo())

print(makoko_gt)
print(makoko_gt.getInfo())

print(mumbai_gt)
print(mumbai_gt.getInfo())

print(kibera_gt)
print(kibera_gt.getInfo())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "users/rhodesailab3/Medellin_s2"
      }
    }
  }
})
{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b5', 'data_t

In [6]:
## reproject & change data type to byte

proj = ee.Projection('EPSG:3857')

medellin_proj = medellin.reproject(proj)
#print(medellin_proj.getInfo())

medellin_gt_proj = medellin_gt.reproject(proj, None, 0.3).byte()
print(medellin_gt_proj.getInfo())

lower_gt_proj = lower_gt.reproject(proj, None, 0.3).byte()

makoko_gt_proj = makoko_gt.reproject(proj, None, 0.3).byte()

mumbai_gt_proj = mumbai_gt.reproject(proj, None, 0.3).byte()

kibera_gt_proj = kibera_gt.reproject(proj, None, 0.3).byte()
print(kibera_gt_proj.getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [4802, 3418], 'origin': [-28064334, -2325853], 'crs': 'EPSG:3857', 'crs_transform': [0.3, 0, 0, 0, -0.3, 0]}], 'version': 1613638906016500.0, 'id': 'users/rhodesailab3/Medellin_ground_truth_s2', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-75.63181296289562, 6.246384348288629], [-75.63180168665014, 6.2463836418232574], [-75.61896555983097, 6.2463988546969], [-75.61892391397645, 6.246435525932105], [-75.61887791704872, 6.2464666389915084], [-75.61887516433639, 6.2464814567165785], [-75.61888581999679, 6.255534715862183], [-75.61892258201927, 6.255576090860556], [-75.61895392740796, 6.255621808007144], [-75.61896511584537, 6.255623852702844], [-75.61897639307902, 6.255624559982755], [-75.63181278531695, 6.255609262767537], [-75.63185442750734, 6.2555726631736315], [-75.63190033157126, 6.255541541627], [-75.63190317282812, 6.25

In [10]:
# determine visualisation parameters
vis_params = {'bands': ['b1'], 'min': 0, 'max': 1, 'palette': ['white', 'blue']}

Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
Map
#Map.addLayer(medellin_gt_proj, vis_params, 'medellin')
#Map.addLayer(lower_gt_proj, vis_params, 'lower')
#Map.addLayer(makoko_gt_proj, vis_params, 'makoko')
#Map.addLayer(mumbai_gt_proj, vis_params, 'mumbai')
Map.addLayer(kibera_gt_proj, vis_params, 'kibera')
Map

## TO DO: why is nothing besides Medellin appearing?!

Map(center=[6.2497, -75.58305], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [ ]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
bands_234 = ['b2', 'b3', 'b4']
# This property of the table stores the land cover labels.
label = 'informal_settlement'

# Overlay the points on the imagery to get training.
training = medellin.select(bands234).sampleRegions(**{
  'collection': polygons,
  'properties': [label],
  'scale': 10
})

In [ ]:
svm_classifier = ee.Classifier.libsvm(kernelType = 'RBF')
trained = svm_classifier.train(training, label, bands)

In [ ]:
result = medellin.select(bands).classify(trained)

In [ ]:
# Display the classification result and the input image.
Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
#Map.addLayer(polygons, {}, 'training polygons')
#Map.addLayer(result, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map

In [ ]:
Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
#Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
Map.addLayer(result, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map.addLayer(polygons, {}, 'training polygons')
Map

In [ ]:
Map.addLayer(polygons, {}, 'training polygons')
Map

In [ ]:
medellin_ground_det={
  "bands": ['b1'],
  "min" : 0,
  "max" : 1
}
Map.addLayer(medellin_gt,medellin_ground_det,'Medellin')
Map

In [ ]:
kianda = ee.Image("users/shehryarsaroya/Kianda_s2")
kianda_gt = ee.Image("users/shehryarsaroya/Kianda_ground_truth_s2")


In [ ]:
result_kianda = kianda.select(bands).classify(trained)

In [ ]:
polygons_kianda = kianda_gt.reduceToVectors(scale = 10, labelProperty = "informal_settlement")

In [ ]:
#print(polygons_kianda.getInfo())

In [ ]:
Map = geemap.Map(center=[-1.3115,36.7659242], zoom=15)
#Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
Map.addLayer(result_kianda, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map.addLayer(polygons_kianda, {}, 'training polygons')
Map